Source: https://platform.openai.com/docs/guides/function-calling/parallel-function-calling

In [ ]:
import json
from typing import Annotated as A, Literal as L

import openai
from pydantic import BaseModel
from annotated_docs import as_json_schema, doc as D, call


client = openai.OpenAI(api_key="")


class Location(BaseModel):
    city: A[str, D("The city, e.g. San Francisco")]
    country: A[str, D("The country, e.g. USA")]

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(
    location: A[Location, D("Location to get the weather for.")],
    unit: L["celsius", "fahrenheit"] = "fahrenheit",
) -> str:
    """Get the current weather in a given location"""
    if "japan" in location.country.lower() and "tokyo" in location.city.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "usa" in location.country.lower() and "san francisco" in location.city.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "france" in location.country.lower() and "paris" in location.city.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

tools = [
    {
        "type": "function",
        "function": as_json_schema(get_current_weather),
    }
]
print(f"tools: {json.dumps(tools, indent=2)}")


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [
        {
            "role": "user",
            "content": "What's the weather like in San Francisco, Tokyo, and Paris?",
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"{function_args=!r}")
            function_response = call(function_to_call, function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


print("")
print(run_conversation().choices[0].message.content)